In [22]:
import os

In [23]:
pwd

'C:\\Users\\dhyey'

[WinError 2] The system cannot find the file specified: '/Desktop'
C:\Users\dhyey\Desktop\Data Science\Coccidiosis-Classification_Project\research


[WinError 3] The system cannot find the path specified: "('\\Desktop\\Data Science\\Coccidiosis-Classification_Project')"
C:\Users\dhyey\Desktop\Data Science\Coccidiosis-Classification_Project\research


In [26]:
os.chdir('\Desktop\Data Science\Coccidiosis-Classification_Project')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\Desktop\\Data Science\\Coccidiosis-Classification_Project'

In [27]:
os.chdir('./Desktop')

In [28]:
os.chdir('./Data Science/Coccidiosis-Classification-Project')

FileNotFoundError: [WinError 2] The system cannot find the file specified: './Data Science/Coccidiosis-Classification-Project'

In [29]:
os.chdir('./Data Science')

In [31]:
os.chdir('./Coccidiosis-Classification_Project')

In [32]:
pwd

'C:\\Users\\dhyey\\Desktop\\Data Science\\Coccidiosis-Classification_Project'

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [56]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [62]:
class ConfigurationManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
             params_filepath = PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


In [63]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [64]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [65]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-25 23:06:47,468: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-25 23:06:47,468: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-25 23:06:47,481: INFO: common: created directory at: artifacts]
[2023-08-25 23:06:47,484: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-25 23:08:34,784: INFO: 884505629: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 278656763
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "1541762e3b2c285b95d6fb0588813be001a36ebe30424180b22c308c43123cab"
Last-Modified: Fri, 25 Aug 2023 17:36:49 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C194:DE32C:65326:843DF:64E8E6B0
Accept-Ranges: bytes
Date: Fri, 25 Aug 2023 17:37:0

In [61]:
config = read_yaml(CONFIG_FILE_PATH)
print("Keys in the configuration:", config.keys())

[2023-08-25 23:04:32,482: INFO: common: yaml file: config\config.yaml loaded successfully]
Keys in the configuration: dict_keys(['artifacts_root', 'data_ingestion'])
